In [ ]:
!pip install ray

     |████████████████████████████████| 57.6 MB 1.3 MB/s 
     |████████████████████████████████| 171 kB 59.5 MB/s 


In [ ]:
!pip install "ray[tune]"

     |████████████████████████████████| 124 kB 25.9 MB/s 


In [ ]:
!pip install raydp

     |████████████████████████████████| 10.5 MB 22.1 MB/s 
     |████████████████████████████████| 1.3 MB 47.5 MB/s 
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 160 kB 43.8 MB/s 
     |████████████████████████████████| 271 kB 59.7 MB/s 
     |████████████████████████████████| 198 kB 68.1 MB/s 
     |████████████████████████████████| 128 kB 62.4 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 192 kB 50.2 MB/s 
     |████████████████████████████████| 3.0 MB 53.8 MB/s 
     |████████████████████████████████| 82 kB 255 kB/s 
     |████████████████████████████████| 201 kB 64.9 MB/s 
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=bea5b6b2f8f162465f59459d27cc07de04b98b403d0d

In [ ]:
!pip install "xgboost_ray"

     |████████████████████████████████| 137 kB 5.0 MB/s 


In [ ]:
%%writefile random_nyctaxi.py
import os
import argparse

import numpy as np
import pandas as pd

base_date = np.datetime64('2010-01-01 00:00:00')

parser = argparse.ArgumentParser(description='Rabdin NYC taxi Generator')
parser.add_argument(
    '--num-records',
    type=int,
    default=2000,
    metavar='N',
    help='number of records to generate (default: 2000)')

args = parser.parse_args()

N = args.num_records

fare_amount = np.random.uniform(3.0, 50.0, size=N)
pick_long = np.random.uniform(-74.2, -73.8, size=N)
pick_lat = np.random.uniform(40.7, 40.8, size=N)
drop_long = np.random.uniform(-74.2, -73.8, size=N)
drop_lat = np.random.uniform(40.7, 40.8, size=N)
passenger_count = np.random.randint(1, 5, size=N)
date = np.random.randint(0, 157680000, size=N) + base_date
date = np.array([t.item().strftime('%Y-%m-%d %H:%m:%S UTC') for t in date])
key = ['fake_key'] * N
df = pd.DataFrame({
    'key': key,
    'fare_amount':fare_amount,
    'pickup_datetime': date,
    'pickup_longitude': pick_long,
    'pickup_latitude': pick_lat,
    'dropoff_longitude': drop_long,
    'dropoff_latitude': drop_lat,
    'passenger_count': passenger_count
    })
csv_path = os.path.dirname(os.path.realpath(__file__)) + '/fake_nyctaxi.csv'
df.to_csv(csv_path, index=False)

Writing random_nyctaxi.py


In [ ]:
!python random_nyctaxi.py

In [ ]:
%%writefile data_process.py
from os.path import dirname, realpath

import numpy as np
from pyspark.sql.functions import *

# change this to where the dataset is
NYC_TRAIN_CSV = 'file://' + dirname(realpath(__file__)) + '/fake_nyctaxi.csv'

def clean_up(data):
    data = data.filter(col('pickup_longitude')<=-72) \
            .filter(col('pickup_longitude')>=-76) \
            .filter(col('dropoff_longitude')<=-72) \
            .filter(col('dropoff_longitude')>=-76) \
            .filter(col('pickup_latitude')<=42) \
            .filter(col('pickup_latitude')>=38) \
            .filter(col('dropoff_latitude')<=42) \
            .filter(col('dropoff_latitude')>=38) \
            .filter(col('passenger_count')<=6) \
            .filter(col('passenger_count')>=1) \
            .filter(col('fare_amount') > 0) \
            .filter(col('fare_amount') < 250) \
            .filter(col('dropoff_longitude') != col('pickup_longitude')) \
            .filter(col('dropoff_latitude') != col('pickup_latitude')) 
    return data

# Add time related features
def add_time_features(data):
    data = data.withColumn("day", dayofmonth(col("pickup_datetime")))
    data = data.withColumn("hour_of_day", hour(col("pickup_datetime")))
    data = data.withColumn("day_of_week", dayofweek(col("pickup_datetime"))-2)
    data = data.withColumn("week_of_year", weekofyear(col("pickup_datetime")))
    data = data.withColumn("month_of_year", month(col("pickup_datetime")))
    data = data.withColumn("quarter_of_year", quarter(col("pickup_datetime")))
    data = data.withColumn("year", year(col("pickup_datetime")))
    
    @udf("int")
    def night(hour, weekday):
        if ((hour <= 20) and (hour >= 16) and (weekday < 5)):
            return int(1)
        else:
            return int(0)

    @udf("int")
    def late_night(hour):
        if ((hour <= 6) and (hour >= 20)):
            return int(1)
        else:
            return int(0)
    data = data.withColumn("night", night("hour_of_day", "day_of_week"))
    data = data.withColumn("late_night", late_night("hour_of_day"))
    return data

def add_distance_features(data):
    @udf("float")
    def manhattan(lat1, lon1, lat2, lon2):
        return float(np.abs(lat2 - lat1) + np.abs(lon2 - lon1))
    
    # Location of NYC downtown
    ny = (-74.0063889, 40.7141667)
    # Location of the three airport in NYC
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    
    # Features about the distance between pickup/dropoff and airport
    data = data.withColumn("abs_diff_longitude", abs(col("dropoff_longitude")-col("pickup_longitude"))) \
               .withColumn("abs_diff_latitude", abs(col("dropoff_latitude") - col("pickup_latitude")))
    data = data.withColumn("manhattan", col("abs_diff_latitude")+col("abs_diff_longitude"))
    data = data.withColumn("pickup_distance_jfk", manhattan("pickup_longitude", "pickup_latitude", lit(jfk[0]), lit(jfk[1])))
    data = data.withColumn("dropoff_distance_jfk", manhattan("dropoff_longitude", "dropoff_latitude", lit(jfk[0]), lit(jfk[1])))
    data = data.withColumn("pickup_distance_ewr", manhattan("pickup_longitude", "pickup_latitude", lit(ewr[0]), lit(ewr[1])))
    data = data.withColumn("dropoff_distance_ewr", manhattan("dropoff_longitude", "dropoff_latitude", lit(ewr[0]), lit(ewr[1])))
    data = data.withColumn("pickup_distance_lgr", manhattan("pickup_longitude", "pickup_latitude", lit(lgr[0]), lit(lgr[1])))
    data = data.withColumn("dropoff_distance_lgr", manhattan("dropoff_longitude", "dropoff_latitude", lit(lgr[0]), lit(lgr[1])))
    data = data.withColumn("pickup_distance_downtown", manhattan("pickup_longitude", "pickup_latitude", lit(ny[0]), lit(ny[1])))
    data = data.withColumn("dropoff_distance_downtown", manhattan("dropoff_longitude", "dropoff_latitude", lit(ny[0]), lit(ny[1])))
    
    return data

def drop_col(data):
    data = data.drop("pickup_datetime") \
            .drop("pickup_longitude") \
            .drop("pickup_latitude") \
            .drop("dropoff_longitude") \
            .drop("dropoff_latitude") \
            .drop("passenger_count") \
            .drop("key")
    return data

def nyc_taxi_preprocess(data):
    data = clean_up(data)
    data = add_time_features(data)
    data = add_distance_features(data)
    return drop_col(data)

if __name__ == '__main__':
    import ray
    import raydp
    ray.init()
    spark = raydp.init_spark("NYCTAXI data processing",
                             num_executors=1,
                             executor_cores=1,
                             executor_memory='500M',
                             configs={"spark.shuffle.service.enabled": "true"})
    data = spark.read.format("csv") \
                     .option("header", "true") \
                     .option("inferSchema", "true") \
                     .load(NYC_TRAIN_CSV)
    # Set spark timezone for processing datetime
    spark.conf.set("spark.sql.session.timeZone", "UTC")
    # Transform the dataset
    data = nyc_taxi_preprocess(data)

Writing data_process.py


In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 1
cores_per_executor = 1
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

2022-01-16 15:39:57,143	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release


Spark Run Time =  1.7844066619873047
Epoch-0: {'num_samples': 1789, 'epoch': 1.0, 'batch_count': 28.0, 'train_loss': 899.6847279326352, 'last_train_loss': 876.887451171875}
Epoch-1: {'num_samples': 1789, 'epoch': 2.0, 'batch_count': 28.0, 'train_loss': 895.1097870299508, 'last_train_loss': 843.1162719726562}
Epoch-2: {'num_samples': 1789, 'epoch': 3.0, 'batch_count': 28.0, 'train_loss': 889.6506001369589, 'last_train_loss': 796.5059204101562}
Epoch-3: {'num_samples': 1789, 'epoch': 4.0, 'batch_count': 28.0, 'train_loss': 882.1949483019604, 'last_train_loss': 792.7432250976562}
Epoch-4: {'num_samples': 1789, 'epoch': 5.0, 'batch_count': 28.0, 'train_loss': 871.5343155410451, 'last_train_loss': 924.8529052734375}
Epoch-5: {'num_samples': 1789, 'epoch': 6.0, 'batch_count': 28.0, 'train_loss': 857.8564123214458, 'last_train_loss': 743.6755981445312}
Epoch-6: {'num_samples': 1789, 'epoch': 7.0, 'batch_count': 28.0, 'train_loss': 841.239653432216, 'last_train_loss': 687.090576171875}
Epoch-7

In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 1
cores_per_executor = 4
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

2022-01-16 15:53:19,113	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265


(scheduler +13m45s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +13m45s) Error: No available node types can fulfill resource request {'CPU': 4.0, 'memory': 524288000.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +14m20s) Error: No available node types can fulfill resource request {'memory': 524288000.0, 'CPU': 4.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +14m55s) Error: No available node types can fulfill resource request {'CPU': 4.0, 'memory': 524288000.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +15m30s) Error: No available node types can fulfill resource request {'memory': 524288000.0, 'CPU': 4.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +16m6s) Error: No available node types can fulfill resource request {'CPU': 4.0, 'memory': 524288000.0}. Add suitable node types to this cl

In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 1
cores_per_executor = 4
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

ModuleNotFoundError: ignored

In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 1
cores_per_executor = 8
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
# raydp.stop_spark()
# ray.shutdown()
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 2
cores_per_executor = 1
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

2022-01-16 13:25:18,618	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please cons

Spark Run Time =  1.1537659168243408
(scheduler +58s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +58s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2022-01-16 13:26:19,690	WARNING worker.py:1245 -- The actor or task with ID ffffffffffffffff0ea38786fea00a4b7905828a01000000 cannot be scheduled right now. You can ignore this message if this Ray cluster is expected to auto-scale or if you specified a runtime_env for this actor or task, which may take time to install.  Otherwise, this is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increasing the resources available to this Ray cluster.
Required resources for this actor or task: {GPU: 1.000000}, {CPU: 1.000000}
Available resources on this node: {0.000000/2.000000 CPU, 320050000.000000 GiB/371250000.000000 GiB memory, 1.000000/1.000000 GPU, 185625000.000000 GiB/185625000.000000 GiB object_store_memory, 1.000000/1.000000 node:172.28.0.2, 1.000000/1.000000 accelerator_type:T4}
 In total there are 0 pending tasks and 1 pending actors on this node.


(scheduler +1m33s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +2m8s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +2m43s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +3m18s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding m

In [ ]:
import ray
import torch
import torch.nn as nn
import torch.nn.functional as F

import raydp
from raydp.torch import TorchEstimator
from raydp.utils import random_split
from time import *

from data_process import nyc_taxi_preprocess, NYC_TRAIN_CSV

# Firstly, You need to init or connect to a ray cluster. Note that you should set include_java to True.
# For more config info in ray, please refer the ray doc. https://docs.ray.io/en/latest/package-ref.html
# ray.init(address="auto")
ray.init()

# After initialize ray cluster, you can use the raydp api to get a spark session
app_name = "NYC Taxi Fare Prediction with RayDP"
num_executors = 4
cores_per_executor = 1
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

# Then you can code as you are using spark
# The dataset can be downloaded from https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data
# Here we just use a subset of the training data
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load(NYC_TRAIN_CSV)
# Set spark timezone for processing datetime
spark.conf.set("spark.sql.session.timeZone", "UTC")
# Transform the dataset
spark_begin_time = time()
data = nyc_taxi_preprocess(data)
spark_end_time = time()
spark_run_time = spark_end_time-spark_begin_time
print("Spark Run Time = ", spark_run_time)

# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.9, 0.1], 0)
features = [field.name for field in list(train_df.schema) if field.name != "fare_amount"]
# Define a neural network model
class NYC_Model(nn.Module):
    def __init__(self, cols):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        
        return x

nyc_model = NYC_Model(len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(nyc_model.parameters(), lr=0.001)
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=1, model=nyc_model, optimizer=optimizer, loss=criterion,
                           feature_columns=features, label_column="fare_amount", batch_size=64,
                           num_epochs=30)
# Train the model
ray_begin_time = time()
estimator.fit_on_spark(train_df, test_df)
ray_end_time = time()
ray_run_time = ray_end_time-ray_begin_time
print("Ray Run Time = ", ray_run_time)
# shutdown raydp and ray
estimator.shutdown()
raydp.stop_spark()
ray.shutdown()

In [ ]:
!